In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, confusion_matrix, log_loss, precision_recall_curve
import lightgbm as lgb
import sys
from pathlib import Path

ROOT_DIR = Path.cwd().parents[0]
sys.path.append(str(ROOT_DIR))

from src.preprocessing import preprocess_data


# loading our train data

train_path = ROOT_DIR / "data" / "raw" / "train.csv"

train_data = pd.read_csv(train_path, index_col='id')

# taking our target column as y
y = train_data['diagnosed_diabetes']
X = train_data.drop(columns="diagnosed_diabetes")

# print(X.head(5))

# extracting categorical and numkerical columns

cat_cols = [col for col in X.columns if X[col].dtype == 'object']

num_cols = [col for col in X.columns if X[col].dtype == 'int64' or X[col].dtype == 'float64']

#print(cat_cols, num_cols)

#Splitting our data into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=1)
#print(X_train.shape, "\n", X_valid.shape)

#Creatin A column Tranformer for preprocessing the data

ordinal_cat_cols = ["education_level", "income_level"]
nominal_cat_cols = ["gender", "ethnicity","smoking_status","employment_status"]


In [7]:
# Buildin a pipeline for LightGBM
processing = preprocess_data(num_cols,ordinal_cat_cols, nominal_cat_cols)

lgb_model = lgb.LGBMClassifier(objective="binary",
    n_estimators=800,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=1,
    n_jobs=-1)

lgb_pipeline = Pipeline(steps=[("preprocessing", processing),
                               ("lgb_model", lgb_model)])

lgb_pipeline.fit(X_train, y_train)

lgb_pred = lgb_pipeline.predict_proba(X_valid)[:,1]
print("Done")

[LightGBM] [Info] Number of positive: 348829, number of negative: 211171
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.107934 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1653
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.622909 -> initscore=0.501914
[LightGBM] [Info] Start training from score 0.501914


d:\practice\diabetes_risk_prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Done


In [8]:
# Calculating metrics of lgb_model
roc_auc_lgb = roc_auc_score(y_valid, lgb_pred)
ll_lgb = log_loss(y_valid, lgb_pred)

print("Roc_Auc_score of LightGBM: ", roc_auc_lgb)
print("Log Loss score of LightGBM: ", ll_lgb)

Roc_Auc_score of LightGBM:  0.7241550583066454
Log Loss score of LightGBM:  0.58425245420171


In [9]:
# Predicting the final results without tuning
test_data = pd.read_csv("../data/raw/test.csv", index_col='id')

lgb_pipeline.fit(X, y)

final_pred = lgb_pipeline.predict_proba(test_data)[:,1]


[LightGBM] [Info] Number of positive: 436307, number of negative: 263693
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1655
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623296 -> initscore=0.503561
[LightGBM] [Info] Start training from score 0.503561


d:\practice\diabetes_risk_prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [11]:
#Saving data as csv file
result = pd.DataFrame({"id": test_data.index ,
                       "diagnosed_diabetes": final_pred})

result.to_csv("../submissions/submission_lgb.csv", index=False)

In [16]:
# Tuning the parameters of the model
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    "lgb_model__learning_rate" :[0.05, 0.1, 0.2],
    "lgb_model__num_leaves" : [31, 60, 80, 100],
    "lgb_model__min_child_samples" : [40, 60, 80],
    "lgb_model__feature_fraction": [0.7, 0.8, 0.9],
    "lgb_model__bagging_fraction": [0.7, 0.8, 0.9]
}

search = RandomizedSearchCV(lgb_pipeline, param_distributions= parameters, cv =3, n_jobs=-1, scoring='roc_auc', random_state=1, n_iter=20, verbose=1)

search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] Number of positive: 348829, number of negative: 211171
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.152936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1653
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 31
[LightGBM] [Info] [binary:Boos

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...sample=0.8))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'lgb_model__bagging_fraction': [0.7, 0.8, ...], 'lgb_model__feature_fraction': [0.7, 0.8, ...], 'lgb_model__learning_rate': [0.05, 0.1, ...], 'lgb_model__min_child_samples': [40, 60, ...], ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",20
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` fo

In [17]:
best_params = search.best_params_
best_score = search.best_score_

print("Best Params - ", best_params)
print("Best Score - ", best_score)

Best Params -  {'lgb_model__num_leaves': 31, 'lgb_model__min_child_samples': 80, 'lgb_model__learning_rate': 0.05, 'lgb_model__feature_fraction': 0.7, 'lgb_model__bagging_fraction': 0.9}
Best Score -  0.726254041518926


In [19]:
#Best Params Model
best_model = search.best_estimator_

best_pred = best_model.predict_proba(X_valid)[:,1]

print("Best Roc Auc score : ", roc_auc_score(y_valid, best_pred))
print("Best Log Loss score : ", log_loss(y_valid, best_pred))


d:\practice\diabetes_risk_prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
Best Roc Auc score :  0.7245475219785746
Best Log Loss score :  0.5839293437201284


In [20]:
# Predicting on test data finally
best_model.fit(X, y)
tuned_pred = best_model.predict_proba(test_data)[:,1]

tuned_result = pd.DataFrame({"id" : test_data.index,
                             "diagnosed_diabetes" : tuned_pred})

tuned_result.to_csv("../submissions/tuned_lgb.csv", index=False)

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] Number of positive: 436307, number of negative: 263693
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080456 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1655
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623296 -> initscore=0.503561
[LightGBM] [

d:\practice\diabetes_risk_prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9
